In [ ]:
from selenium import webdriver
import time
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By 
import os
import pandas as pd
import datetime
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait as Wait

tax = 0.05#股市手续费
interest = 0.115#收益,高于则脱售
reduction_ratio = 0.118#低于昨日闭市报则购买单位
zb_lim = 40#保留zb数量
global have_buy_num 
have_buy_num=20
price_limits = 0.1#低于这个价格的股票不购买

def open_url():
    driver = uc.Chrome()
    #driver = webdriver.Edge(r'C:\Users\Administrator\MLtest\zod\driver\msedgedriver.exe')
    driver.get('https://zodgame.xyz/plugin.php?id=jninvest')
    print('打开浏览器成功')
    
    #自行添加下面所需的cookies
    cookie_zod = []
    cookie_zod.append({"name":"cf_clearance","value":""})
    cookie_zod.append({"name":"qhMq_2132_auth","value":""})
    cookie_zod.append({"name":"qhMq_2132_saltkey","value":""})
    cookie_zod.append({"name":"qhMq_2132_sid","value":""})

    #cookie_zod.append({"name":"qhMq_2132_lip","value":""})
    #cookie_zod.append({"name":"qhMq_2132_ulastactivity","value":""})
    #cookie_zod.append({"name":"qhMq_2132_lastact","value":""})
    #cookie_zod.append({"name":"qhMq_2132_lastcheckfeed","value":""})
    #cookie_zod.append({"name":"qhMq_2132_lastvisit","value":""})
    #cookie_zod.append({"name":"qhMq_2132_myrepeat_rr","value":""})
    #cookie_zod.append({"name":"qhMq_2132_smile","value":""})
    
    time.sleep(10)
    for i in range(len(cookie_zod)):
        print('cookies注入',cookie_zod[i])
        driver.add_cookie(cookie_zod[i])
    print('cookies注入完成')
    driver.refresh()
    print('刷新浏览器')
    time.sleep(20)
    return driver

In [ ]:
def get_time():
    now_time = datetime.datetime.now()
    if now_time.minute >= 40:
        minute = 40
    elif now_time.minute >= 20:
        minute = 20
    else:
        minute = 0
    n_time = str(now_time.year)+'.'+str(now_time.month)+'.'+str(now_time.day)+'.'+str(now_time.hour)+'.'+str(minute)
    return n_time,now_time.minute

In [ ]:
def get_shareslist():
    #获得股票信息
    shares1 = driver.find_element(By.XPATH,'//*[@id="profilelist"]/tbody/tr[2]')
    shares2 = driver.find_element(By.XPATH,'//*[@id="profilelist"]/tbody/tr[3]')
    shares3 = driver.find_element(By.XPATH,'//*[@id="profilelist"]/tbody/tr[4]')
    shares4 = driver.find_element(By.XPATH,'//*[@id="profilelist"]/tbody/tr[5]')
    shares5 = driver.find_element(By.XPATH,'//*[@id="profilelist"]/tbody/tr[6]')
    shares6 = driver.find_element(By.XPATH,'//*[@id="profilelist"]/tbody/tr[7]')

    #标准化
    shareslist =[]
    shares1 = str(shares1.text).replace('[', '').replace(']', '').replace('\n', ' ').replace('+', '').split()
    shareslist.append(shares1)
    shares2 = str(shares2.text).replace('[', '').replace(']', '').replace('\n', ' ').replace('+', '').split()
    shareslist.append(shares2)
    shares3 = str(shares3.text).replace('[', '').replace(']', '').replace('\n', ' ').replace('+', '').split()
    shareslist.append(shares3)
    shares4 = str(shares4.text).replace('[', '').replace(']', '').replace('\n', ' ').replace('+', '').split()
    shareslist.append(shares4)
    shares5 = str(shares5.text).replace('[', '').replace(']', '').replace('\n', ' ').replace('+', '').split()
    shareslist.append(shares5)
    shares6 = str(shares6.text).replace('[', '').replace(']', '').replace('\n', ' ').replace('+', '').split()
    shareslist.append(shares6)
    return shareslist


In [ ]:
#历史价格写入csv存储
def write_csv(shareslist,if_frist = 0):
    if if_frist == 1:
        shares_df = [0]
        shares_df = pd.DataFrame(shares_df)

        for i in range(len(shareslist)):
            a = str(shareslist[i][0])
            if a not in shares_df.index:
                shares_df .loc[a] = 0
        now_time,_ = get_time()
        shares_df.insert(shares_df.shape[1],now_time,0,allow_duplicates=True)#插入时间列

        #删除多的一行和一列
        shares_df.drop(shares_df.index[0],axis=1,inplace=True)
        shares_df.drop(shares_df.index[0], inplace=True)
        #print(shares_df)
    else:
        shares_df = pd.read_csv("zodshares.csv",encoding='gbk',index_col=0) 
        for i in range(len(shareslist)):
            a = str(shareslist[i][0])
            if a not in shares_df.index:
                shares_df .loc[a] = 0
        now_time,_ = get_time()
        shares_df.insert(shares_df.shape[1],now_time,0,allow_duplicates=True)#插入时间列


    for j in range(len(shareslist)):
        shares_name = shareslist[j][0]
        x = list(shares_df.index).index(shares_name)
        y = shares_df.shape[1]-1
        #print(shareslist[j][3])
        shares_df.iloc[x,y]=shareslist[j][3]
    
    if(os.path.isfile("./zodshares.csv")):
        os.remove('./zodshares.csv')
    shares_df.to_csv('zodshares.csv',index=True,encoding='gbk')
    print('写入csv文件成功\n')
    return shares_df


In [ ]:
def get_purchased_shares():
    full_xpath = '/html/body/div[5]/div[2]/table/tbody/tr[2]/td[1]/div[6]/div/div[2]/table/tbody'
    a = driver.find_element(By.XPATH,full_xpath).find_elements(By.TAG_NAME, "tr")
    arr = []
    arr1 = []
    for tr in a:
        arr1 = (tr.text).replace('[', '').replace(']', '').replace('\n', ' ')
        arr1 =arr1.replace('+', '').replace('当前价格 (每股)', '当前价格每股').split(" ")
        arr.append(arr1)
    return arr


In [ ]:
def sale_shares():
    fxpath = '/html/body/div[5]/div[2]/table/tbody/tr[2]/td[1]/div[6]/div/div[2]/table'#已买股票的表格xpath
    #sale_xpath ='/html/body/div[5]/div[2]/table/tbody/tr[2]/td[1]/div[6]/div/div[2]/table/tbody/tr[2]/td[6]/a/font'
    shares = driver.find_element(By.XPATH,fxpath)
    if '暂时没有'in shares.text:
        print('目前手上并没有股票')
    else:
        purchased_shares_list = get_purchased_shares()
        j = 0
        for i in range(len(purchased_shares_list)-1):
            purchased_shares_list = get_purchased_shares()
            print('检测',purchased_shares_list[j+1][0],'是否可以脱售')
            if float(purchased_shares_list[j+1][-2])*(1+tax+interest)<=float(purchased_shares_list[j+1][2]):
                print('可以脱售',purchased_shares_list[j+1][0])
                sale_xpath='/html/body/div[5]/div[2]/table/tbody/tr[2]/td[1]/div[6]/div/div[2]/table/tbody/tr['+str(j+2)+']/td[6]/a/font'
                driver.implicitly_wait(5)
                time.sleep(3)
                comfirmdel=driver.find_element(By.XPATH,sale_xpath)
                driver.execute_script("arguments[0].click();", comfirmdel)
                
                #driver.find_element(By.XPATH,sale_xpath).click()#点击脱售
                try:
                    Wait(driver, 2*60).until(EC.presence_of_element_located((By.ID, "fwin_dialog_submit")))#阻塞等待确认界面加载
                except:
                    print('\n!!!!未知原因无法脱售该股票,已终止本次股票脱售\n!!!!')
                    return 1
                driver.find_element(By.XPATH,'//*[@id="fwin_dialog_submit"]/strong').click()#脱售确认的xpath
                print('脱售成功')
                print('认购股份平均价格:',purchased_shares_list[j+1][-2])
                print('当前价格 (每股):',purchased_shares_list[j+1][2])
                get_money = 0.95*(float(purchased_shares_list[j+1][2])-float(purchased_shares_list[j+1][-2]))*float(purchased_shares_list[j+1][-3])
                print('本次交易中,扣除手续费赚取了',get_money,'ZB\n')
                j=i
                time.sleep(20)#等待自动刷新
            else:
                print('不满足脱售条件\n')
                j=i+1
                
                
            
    

In [ ]:
def buy_shares(i,num_buy):
    buy_xpath = '/html/body/div[5]/div[2]/table/tbody/tr[2]/td[1]/div[4]/div/div[2]/table/tbody/tr['+str(i+1)+']/td[10]/a'
    Wait(driver, 5*60).until(EC.presence_of_element_located((By.XPATH, buy_xpath)))
    time.sleep(10)
    driver.find_element(By.XPATH,buy_xpath).click()#点击购买
    time.sleep(40)
    '''
    try:
        Wait(driver, 2*60).until(EC.presence_of_element_located((By.ID, "paymount")))#阻塞等待加载页面
    except:
        print('\n!!!!未知原因无法购买股票,已终止本次股票购买\n!!!!')
        return'''
    #options = driver.find_elements_by_tag_name('option')
    options = driver.find_elements(By.TAG_NAME,'option')
    #if num_buy>=6:
        #num_buy = 6
    print('买多少手',num_buy)
    options[num_buy].click() #1-5*100股 ,6 1000股
    yes_xpath = '/html/body/div[1]/div/table/tbody/tr[2]/td[2]/form/p/button/strong'
    driver.find_element(By.XPATH,yes_xpath).click()
    time.sleep(35)
    try:
        if '已经卖光'in driver.find_element(By.XPATH,'/html/body/div[1]/div[2]/table/tbody/tr[2]/td[2]/div').text:
            print('股票已经卖光,购买失败')
    except:
        print('购买成功,等待网页自动刷新')
        global have_buy_num 
        if num_buy>=5:
            have_buy_num  = have_buy_num - num_buy
        if num_buy ==6:
            have_buy_num  = have_buy_num - 10
    time.sleep(15)#等待网页自动刷新

In [ ]:
def shares_if_buy():
    num_buy = 1
    zb_xpath = '//*[@id="wp"]/div[2]/table/tbody/tr[1]/td[3]/div[2]/div/div[2]/ul/li[8]'
    zb_num = int(driver.find_element(By.XPATH,zb_xpath).text.split()[1])#获取zb的数量
    if zb_num<= zb_lim:
        print('ZB数量少与',zb_lim,'不执行购买操作,你破产的风险较高\n.')
    else:
        for i in range(len(shareslist)):
            num_lim = int(shareslist[i][4])/100
            if num_lim>=10:#最多买几手,再这里修改参数进行限制.
                num_lim=10
            if int(shareslist[i][4])==0:
                print(shareslist[i][0],'股票数量为0,跳过检测')
                if i+1==len(shareslist):
                    print('本轮购买结束\n等待下一轮开始\n')
                continue
            else:
                print(1-(float(shareslist[i][3])/float(shareslist[i][6])))
                if 1-(float(shareslist[i][3])/float(shareslist[i][6]))>=reduction_ratio:
                    print(shareslist[i][0],'可以购买,应该能赚')
                    
                    zb_num = int(driver.find_element(By.XPATH,zb_xpath).text.split()[1])#获取zb的数量
                    for num in range(10):
                        if zb_num-(float(shareslist[i][3])*100*(num+1))>=40 and num+1<=num_lim:
                            num_buy = num+1
                            if num_buy>=6 and num_buy<10:
                                num_buy=5
                            elif num_buy>=10:
                                num_buy=6
                                
                    if have_buy_num < num_buy:
                        num_buy = have_buy_num
                        
                    if zb_num-(float(shareslist[i][3])*100*num_buy)>=40:
                        print('正在购买',num_buy,'手股票,预计花费',float(shareslist[i][3])*100*num_buy,'ZB')
                        if float(shareslist[i][3]) >= price_limits:
                            #print('i=',i)
                            buy_shares(i+1,num_buy)
                        elif float(shareslist[i][3]) < price_limits:
                            print('股票单价过低,赚得少,不买')
                    else:
                        print('购买后剩余',zb_num-(float(shareslist[i][3])*100*num_buy),'ZB,不足',zb_lim,'ZB,放弃购买')
            if i+1==len(shareslist):
                print('本轮购买结束\n等待下一轮开始\n')

In [ ]:
if __name__ == '__main__':#主循环
    global have_buy_num
    driver = open_url()
    shareslist = get_shareslist()#获得股票信息
    shares_df = write_csv(shareslist,if_frist = 1)#

    while 1:
        d_time = 10
        ntime = datetime.datetime.now()
        weekday = ntime.weekday()
        if weekday == 6 or ntime.hour < 9:
            d_time = 60*60
            print('股市已经关闭')
            time.sleep(d_time)
            continue
        if weekday <= 5 or ntime.hour >= 9:
            d_time = 10
        if ntime.hour == 8:#8点重置购买次数重置
            have_buy_num = 10
            print('购买次数重置')
        _,now_minute = get_time()
        if now_minute ==4 or now_minute==22 or now_minute==43 :
            print(datetime.datetime.now())
            driver.get('https://zodgame.xyz/plugin.php?id=jninvest')
            time.sleep(40)
            print('获得股票信息')
            shareslist = get_shareslist()#获得股票信息
            shares_df = write_csv(shareslist,if_frist = 0)#写入csv文件
            print('股票信息已记录到csv文件中\n')
            purchased_shares = get_purchased_shares()
            print("开始脱售股票\n")
            t_if = sale_shares()#脱售股票
            if t_if==1:
                print('重试脱售股票')
                sale_shares()
            print('还能购买',have_buy_num,'次')
            print('检测是否满足购买股票条件\n')
            if have_buy_num > 0:
                shares_if_buy()#购买股票
            elif have_buy_num <= 0:
                print('今天已经无法在购买了')
            time.sleep(3*60)
        time.sleep(d_time)
        